In [ ]:
# Import necessary libraries
!pip install mlflow
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
import mlflow
import mlflow.sklearn

# Question 1: Data Structure and Processing Pipeline
class IrisDataProcessor:
    def __init__(self):
        # Load the Iris dataset
        self.data = load_iris()
        # Convert to DataFrame for easier manipulation
        self.df = pd.DataFrame(self.data.data, columns=self.data.feature_names)
        self.df['target'] = self.data.target
        self.scaler = StandardScaler()

    def prepare_data(self):
        # Scale features and perform train-test split
        X = self.df[self.data.feature_names]
        y = self.df['target']
        X_scaled = self.scaler.fit_transform(X)
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
        return X_train, X_test, y_train, y_test

    def get_feature_stats(self):
        # Return basic statistical analysis
        return self.df.describe()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.1/569.1 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.0 MB/s eta 0:00:00


In [ ]:
# Question 2: Experiment Tracking and Model Development
class IrisExperiment:
    def __init__(self, data_processor):
        self.X_train, self.X_test, self.y_train, self.y_test = data_processor.prepare_data()
        mlflow.set_experiment("Iris Classification Experiment")

    def run_experiment(self):
        # Initialize models
        models = {
            "Logistic Regression": LogisticRegression(),
            "Random Forest": RandomForestClassifier()
        }
        # Track experiments
        for model_name, model in models.items():
            with mlflow.start_run(run_name=model_name):
                # Perform cross-validation
                cv_scores = cross_val_score(model, self.X_train, self.y_train, cv=5)
                model.fit(self.X_train, self.y_train)
                predictions = model.predict(self.X_test)

                # Calculate metricsdocker-compose --version

                accuracy = accuracy_score(self.y_test, predictions)
                precision = precision_score(self.y_test, predictions, average='macro')
                recall = recall_score(self.y_test, predictions, average='macro')

                # Log metrics with MLflow
                mlflow.log_param("Model Type", model_name)
                mlflow.log_metric("CV Accuracy", np.mean(cv_scores))
                mlflow.log_metric("Test Accuracy", accuracy)
                mlflow.log_metric("Precision", precision)
                mlflow.log_metric("Recall", recall)

                # Log the model
                mlflow.sklearn.log_model(model, model_name)

    def log_results(self):
        print("Experiment results have been logged in MLflow.")

In [ ]:
# Question 3: Model Optimization and Testing
class IrisModelOptimizer:
    def __init__(self, experiment):
        self.model = LogisticRegression()
        self.X_train, self.X_test, self.y_train, self.y_test = experiment.X_train, experiment.X_test, experiment.y_train, experiment.y_test

    def quantize_model(self):
        # Simple quantization example: reducing the model to int8 precision if needed
        self.model.fit(self.X_train, self.y_train)
        # Simulated quantization step (normally involves model serialization)
        # In practice, this requires frameworks such as TensorFlow Lite or ONNX.
        print("Model quantization complete.")

    def run_tests(self):
        # Simple unit test
        predictions = self.model.predict(self.X_test)
        accuracy = accuracy_score(self.y_test, predictions)
        assert accuracy > 0.7, "Accuracy should be above 70%"
        print("Unit test passed with accuracy:", accuracy)